# Loading The Dependencies

In [3]:
import tensorflow as tf
import numpy as np

# Loading Data File

In [4]:
import helper

source_path='data/small_vocab_en'
target_path='data/small_vocab_fr'

source_text=helper.load_data(source_path)
target_text=helper.load_data(target_path)

## Exploring Data

In [5]:
print ("Data Set Information")
print ("The Number of Unique Words: %s"%(len({word: None for word in source_text.split()})))

sentence_arr=[]
sentences=source_text.split('\n')
for sentence in sentences:
    sentence_arr.append(sentences)
print ("The Number of Sentences are: %r "%(len(sentence_arr)))


#print ("The Number of Sentences are: %s"%(len(sentence.split()) for sentence in sentences))

#print (word_count)
#print 'The Number of Unique Words %r'(len(word: None for word  )

Data Set Information
The Number of Unique Words: 227
The Number of Sentences are: 137861 


## Implementing Pre-Processing

In [6]:
def text_to_ids(source_text,target_text,source_vocab_to_int,target_vocab_to_int):
    source_text_id=[]
    target_text_id=[]
    
    id_eos=target_vocab_to_int['<EOS>']
    
    for source_sentence in source_text.split('\n'):
        a=[source_vocab_to_int[word] for word in source_sentence.split()]
        source_text_id.append(a)
        
    for target_sentence in target_text.split('\n'):
        b=[target_vocab_to_int[word] for word in target_sentence.split()]
        b.append(id_eos)
        target_text_id.append(b)
    
    return source_text_id,target_text_id
        
    

### Pre-Processing Data and Saving it


In [8]:
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

### CheckPoint
If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [9]:
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

## Builiding Neural Network

### Input


In [11]:
def model_input():
    input_=tf.placeholder(tf.int32,[None,None],name='input')
    targets=tf.placeholder(tf.int32,[None,None])
    learning_rate=tf.placeholder(tf.float32)
    keep_prob=tf.placeholder(tf.float32,name='keep_prob')
    
    return input_,targets,learning_rate,keep_prob

### Processing Decoding Input

In [13]:
def process_decoding_input(target_data,target_vocab_to_int,batch_size):
    remove_last_word=tf.strided_slice(target_data,[0,0],[batch_size,-1],[1,1])
    concat_go=tf.concat([tf.fill([batch_size,1],target_vocab_to_int['<GO>']),remove_last_word],1)
    return concat_go

### Encoding Layer
Implementing Encoder RNN Layer

In [14]:
def encoding_layer(rnn_input,rnn_size,num_layers,keep_prob):
    lstm=tf.contrib.nn.BasicLSTMCell(rnn_size)
    cell=tf.contrib.nn.DropoutWrapper(lstm,output_keep=keep_prob)
    enc_cell=tf.contrib.nn.MultiRNNCell([cell]*num_layers)
    _,enc_state=tf.nn.dynamic_rnn(enc_cell,rnn_inputs,dtype=tf.float32)
    return enc_state

### Decoder Layer Training

In [16]:
def decoding_layer_train(encoder_state,dec_cell,dec_embed_input,sequence_length,decoding_scope,output_fn,keep_prob):
    """
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param sequence_length: Sequence Length
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_fn: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: Train Logits
    """
    decoder_fn=tf.contrib.seq2seq.simple_decoder_fn_train(encoder_state)
    outputs,final_state,final_context_state = tf.contrib.seq2seq.dynamic_rnn_decoder(
        dec_cell,
        decoder_fn,
        inputs=dec_embed_input,
        sequence_length=sequence_length,
        scope=decoding_scope)
    
    outputs= tf.nn.dropout(outputs,keep_prob=keep_prob)
    
    return decoder_fn(outputs)

### Decoding Inference

In [19]:
def decoding_layer_infer(encoder_state,dec_cell,dec_embeddings,start_of_sequence_id,end_of_sequence_id,maximum_length,
                         vocab_size,decoding_scope,output_fn,keep_prob):

    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param maximum_length: The maximum allowed time steps to decode
    :param vocab_size: Size of vocabulary
    :param decoding_scope: TensorFlow Variable Scope for decoding
    :param output_fn: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: Inference Logits
    """
    
    infer_decoder_fn=tf.contrib.seq2seq.simple_decoder_fn_inference(
        output_fn,encoder_state,dec_embeddings,start_of_sequence_id,end_of_sequence_id,maximum_length-1,vocab_size)
    inference_logits,_,_=tf.contrib.seq2seq.dynamic_rnn_decoder(dec_cell,infer_decoder_fn,scope=decoding_scope)
    
    return inference_logits

### Decoding Layer

In [21]:
def decoding_layer(dec_embed_input,dec_embeddings,encoder_state,vocab_size,sequence_length,rnn_size,num_layers,
                   target_vocab_to_int,keep_prob):

    """
    :param dec_embed_input: Decoder embedded input
    :param dec_embeddings: Decoder embeddings
    :param encoder_state: The encoded state
    :param vocab_size: Size of vocabulary
    :param sequence_length: Sequence Length
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param keep_prob: Dropout keep probability
    :return: Tuple of (Training Logits, Inference Logits)
    """
    with tf.variable_scope("decoding") as decoding_scope:
        basic_cell=tf.contrib.rnn.BasicLSTMCell(rnn_size)
        rnn_cell=tf.contrib.rnn.MultiRNNCell([basic_cell]*num_layers)
        
        output_fn=lambda x: tf.contrib.layers.fully_connected(
            x,
            vocab_size,
            activation_fn=None,
            scope=decoding_scope)
        
        training_logits=decoding_layer_train(
            encoder_state,
            rnn_cell,
            dec_embed_input,
            sequence_length,
            decoding_scope,
            output_fn,
            keep_prob)
        
        decoding_scope.reuse_variables()
        
        inference_logits = decoding_layer_infer(
            encoder_state,
            rnn_cell,
            dec_embeddings,
            target_vocab_to_int['<GO>'],
            target_vocab_to_int['<EOS>'],
            sequence_length,
            vocab_size,
            decoding_scope,
            output_fn,
            keep_prob)
        
    return training_logits, inference_logits